# Проектный практикум 3

Создать нейронную сеть, способную генерировать текстовые отзывы о различных местах на основе определенных входных параметров, таких как категория места, средний рейтинг и ключевые слова.

## 1. Исследовательский анализ данных

In [1]:
!pip install transformers

In [2]:
pip install torch

In [3]:
from transformers import pipeline

In [4]:
import pandas as pd
import spacy
import re

In [5]:
# Путь к вашему файлу .tskv
file_path = 'geo-reviews-dataset-2023.tskv'

In [6]:
# Список для хранения данных
data = []

# Чтение файла построчно
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Удаляем пробелы и символы новой строки
        line = line.strip()
        if line:  # Проверяем, что строка не пустая
            # Разделяем строку на пары "ключ=значение"
            items = line.split('\t')  # tskv использует табуляцию как разделитель
            data_dict = {}
            for item in items:
                key, value = item.split('=', 1)  # Разделяем только по первому '='
                data_dict[key] = value
            data.append(data_dict)

# Создание DataFrame из списка словарей
df = pd.DataFrame(data)

# Вывод первых нескольких строк DataFrame
df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  500000 non-null  object
 1   name_ru  499030 non-null  object
 2   rating   500000 non-null  object
 3   rubrics  500000 non-null  object
 4   text     500000 non-null  object
dtypes: object(5)
memory usage: 19.1+ MB


In [8]:
# Анализ рубрик
df.rubrics.value_counts()

,rubrics
Гостиница,42242
Ресторан,14615
Кафе,12366
Супермаркет,8899
Магазин продуктов,5289
...,...
Технические и медицинские газы;Сварочные работы;Кованые изделия,1
"Ресторан;Бар, паб;Кафе;Столовая",1
Буровые работы;Нефтегазовая компания,1
Детский магазин;Детская мебель;Пункт выдачи,1


In [9]:
# Разделение рубрик по разделителю и создание новых строк
df['rubrics'] = df['rubrics'].str.replace(';', ',')  # Сначала заменяем ';' на ','
df['rubrics'] = df['rubrics'].str.split(',')  # Теперь разбиваем по ','
df = df.explode('rubrics')

df.rubrics.value_counts()

,rubrics
Кафе,58496
Ресторан,56761
Гостиница,43133
Магазин продуктов,21346
Супермаркет,19746
...,...
Чистка и ремонт колодцев,1
Производство кормов для домашних животных,1
Геральдика и генеалогия,1
Транспортная инфраструктура,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 998606 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  998606 non-null  object
 1   name_ru  996495 non-null  object
 2   rating   998606 non-null  object
 3   rubrics  998606 non-null  object
 4   text     998606 non-null  object
dtypes: object(5)
memory usage: 45.7+ MB


In [11]:
# Поиск дубликатов
duplicates = df.duplicated(subset=['text'])

num_duplicates = duplicates.sum()
print(f"Количество дубликатов: {num_duplicates}")

Количество дубликатов: 498744


In [12]:
df = df.drop_duplicates(subset=['text'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499862 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  499862 non-null  object
 1   name_ru  498895 non-null  object
 2   rating   499862 non-null  object
 3   rubrics  499862 non-null  object
 4   text     499862 non-null  object
dtypes: object(5)
memory usage: 22.9+ MB


In [14]:
# Анализ распределения целевых классов
df.rating.value_counts()

,rating
5.,390383
4.,41154
1.,34351
3.,21686
2.,12088
0.,200


In [15]:
# Очистка рейтинга от точек и преобразование в целые числа
df['rating'] = df['rating'].str.replace('.', '', regex=False).astype(int)

In [16]:
df.rating.value_counts()

,rating
5,390383
4,41154
1,34351
3,21686
2,12088
0,200


Потребители, по всей видимости, склонны оставлять позитивные отзывы.

In [17]:
number_unique_name = df.name_ru.nunique()
print('Количество уникальных наименований организаций: {}'.format(number_unique_name))

Количество уникальных наименований организаций: 148442


In [18]:
number_unique_address = df.address.nunique()
print('Количество уникальных адресов организаций: {}'.format(number_unique_address))

Количество уникальных адресов организаций: 191869


Уникальных адресов на сорок две с лишним тысячи бальше чем уникальных ораганизаций. Это означает, что некоторые организации работают по нескольким адресам. По каждому адресу могут быть различные отзывы потребителей.

In [19]:
number_unique_rubrics = df.rubrics.nunique()
print('Количество уникальных рубрик: {}'.format(number_unique_rubrics))

Количество уникальных рубрик: 1253


In [20]:
number_unique_text = df.text.nunique()
print('Количество уникальных текстов: {}'.format(number_unique_text))

Количество уникальных текстов: 499862


In [21]:
text_by_name = df.groupby('name_ru')['text'].agg(
    ['count']
).sort_values(by='count', ascending=False)

text_by_name

,count
name_ru,
Пятёрочка,6026
Магнит,2609
Красное&Белое,1731
Wildberries,1692
Ozon,1492
...,...
Лингва Плюс,1
VIP шашлык,1
Линарис,1


In [22]:
text_by_address_name = df.groupby(['address', 'name_ru'])['text'].agg(
    ['count']
).sort_values(by='count', ascending=False)

text_by_address_name

count
address                                            name_ru                                                  
Москва, проспект Андропова, 1                      Остров мечты                                          226
Москва, Большая Грузинская улица, 1с1              Московский зоопарк                                    162
Краснодарский край, городской округ Сочи, аэроп... Международный аэропорт Сочи имени В. И. Севасть...    155
Краснодар, Городской сад                           Парк Краснодар                                        151
Москва, Голубинская улица, 16                      Мореон                                                143
...                                                                                                      ...
Москва, улица Кирпичные Выемки, 2, корп. 1         Бигарден                                                1
                                                   Власть взгляда                                          1
                                                   Элна-мебель                                             1
Москва, улица Кирпичные Выемки, 2к1                Ozon                                                    1
​М-5 1705 километр, 2а пос. Березовый Мост, Сат... Лесная жемчужина                                        1

[275195 rows x 1 columns]

In [23]:
text_by_rubrics = df.groupby('rubrics')['text'].agg(
    ['count']
).sort_values(by='count', ascending=False)

text_by_rubrics.head(20)

,count
rubrics,
Гостиница,42664
Ресторан,39793
Кафе,31274
Супермаркет,14373
Салон красоты,11951
Магазин продуктов,11790
Быстрое питание,9632
Торговый центр,8071
Музей,8048


Отзывы потребители в основном оставляют в сферах гостиничных услуг, общественного питания, торговли, индустрии красоты.

## Выводы по разделу

- в среднем на одну организацию приходится чуть более 3 отзывов.
- рубрик очень много, что затрудняет анализ данных и может снизить качество генерации текста. При этом многое рубрики пересекаются. По всей видимости их лучше разметить вручную.

Для генерации текстов о местах оказания услуг и продажи товаров проверим следующий подход. Поскольку приложение создается для потребителей, оно максимально должно быть для них полезным. Что интересует потребителя? Прежде всего товар или услуга (в нашем случае рубрика), место ее оказания, поставки и качество поставщика. Исходя из этого создадаим последовательные фильтры оптимизации поиска: фильтр по рубрике - фильтр по городу - фильтр по улице - фильтр по рейтингу. После фильтрации появится список организаций, по которым пользователь, с помощью нейронной сети получит характеристику организаций.
Протестируем модель facebook/bart-large-cnn.

## 2. Оптимизация поиска и тестирование модели (без предобработки текстов отзывов)

In [24]:
# Ввод названия города пользователем
city_name = input("Введите название города: ")

# Поиск совпадений по городу в колонке address
matching_rows = df[df['address'].str.contains(city_name, case=False, na=False)]

# Проверка на наличие найденных совпадений
if not matching_rows.empty:
    # Ввод названия улицы пользователем
    street_name = input("Введите название улицы: ")

    # Фильтрация по выбранной улице
    street_filtered = matching_rows[matching_rows['address'].str.contains(street_name, case=False, na=False)]

    # Проверка на наличие найденных совпадений по улице
    if not street_filtered.empty:
        # Ввод рубрики пользователем
        rubric_name = input("Введите название рубрики: ")

        # Фильтрация по выбранной рубрике
        rubric_filtered = street_filtered[street_filtered['rubrics'].str.contains(rubric_name, case=False, na=False)]

        # Проверка на наличие найденных совпадений по рубрике
        if not rubric_filtered.empty:
            # Ввод минимального рейтинга пользователем
            min_rating = float(input("Введите минимальный рейтинг (например, 4.0): "))

            # Фильтрация по рейтингу
            final_results = rubric_filtered[rubric_filtered['rating'] >= min_rating]

            # Проверка на наличие найденных совпадений по рейтингу
            if not final_results.empty:
                # Вывод таблицы с нужными столбцами
                result = final_results['name_ru'].unique()
                print(result)
            else:
                print(f"В городе '{city_name}' на улице '{street_name}' нет организаций в рубрике '{rubric_name}' с рейтингом не ниже {min_rating}.")
        else:
            print(f"В городе '{city_name}' на улице '{street_name}' нет организаций в рубрике '{rubric_name}'.")
    else:
        print(f"В городе '{city_name}' нет организаций на улице '{street_name}'.")
else:
    print(f"Город '{city_name}' не найден в адресах.")

Введите название города: тюмень
Введите название улицы: республики
Введите название рубрики: ресторан
Введите минимальный рейтинг (например, 4.0): 4
['Кацо' 'Casa mia' 'Счастье живет здесь' 'Тсуру' 'Своя компания'
 'Легкий чек' 'Отпуск' '150 Meters' 'Plovlounge' 'Трактир ЕрмолаевЪ']


In [27]:
# Инициализация модели для суммаризации BART
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Проверка на наличие найденных совпадений по рейтингу перед суммаризацией
if not final_results.empty:
    # Агрегируем отзывы по каждой организации
    aggregated_reviews = final_results.groupby('name_ru')['text'].apply(lambda x: " ".join(x)).reset_index()

    # Суммаризация отзывов для каждой организации
    for index, row in aggregated_reviews.iterrows():
        text_to_summarize = row['text']
        # Обратите внимание на параметры max_length и min_length
        summary = summarizer(text_to_summarize, max_length=1000, min_length=40, do_sample=False)
        print(f"Организация: {row['name_ru']}")
        print(f"Суммаризация отзыва: {summary[0]['summary_text']}\n")
else:
    print(f"Нет организаций для суммаризации.")

Your max_length is set to 1000, but your input_length is only 814. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=407)
Your max_length is set to 1000, but your input_length is only 282. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=141)


Организация: 150 Meters
Суммаризация отзыва:  ограничени   фиксировано,   Не  тем более дойти,  ‘з-за этого ще тренд с о  раз час хейне. На  Наприятная   “ак типа”,  ‘‘’’, “’щ.’ ’Н. ’ ’. ‘Љ. “Н,’  ’,”  ””.

Организация: Casa mia
Суммаризация отзыва:  приятная,    “пиццы,  чай  глинтвейн. И  разно  течер, тантернее, регент, щерпрект, оперенне, ‘па  тенет,’   “Татьяне’, “”, ‘‘””Н  , ”’”.” “ Н‚’е  хращи,”  ‘Не тракте,  “,“,  ”;. “ ”: “щ.’ ”.  ,   . ”,  “т.  т.  а”,. “а’ ’Љ’. ‚ Љ.  ’а   мен ютне   манеджер’,.  е ч.  непнерар,‚. ‘т.Н. т’: ‘ мнещ  ранщща, т. ’ .

Организация: Plovlounge
Суммаризация отзыва:  очень    оперативно  цены. Не щеню  рещирное, чай   общща  тейна,  ‘Непреганте,’ ‘‘’’,  ‘, ‘;.’;. ‘. ’, “”, ’”,. ‘ “,”. “,  ’.”., ‘,.  , ”’,. ”,‘, . ,. “;’.,  .  ,. ,   “Н.     ””;.  ”.‚’: “ Н. отзывчив’ , “ ”: ‘Ка. хра.тре.щ. тр.х. приготомен, ’е т.  ренищ муйщ фене раказани, р. риненагер,  ме.Н, та. манер   ’партен  т. п. Капинка твëрдр.Мент, На.К. Ме. нащкреретне, п.  янечна    неМара

IndexError: index out of range in self

Суммаризация крайне некачественная. Требуется предобработка текстов отзывов. Верификация и устранению ошибок. Также на следующем этапе планируем попробовать несколько моделей.


## 3. Предобработка текстов отзывов (этот раздел пока не завершен)

In [ ]:
# Вывод строк с индексами от 15 до 23
for i in range(10, 34):
    print(df.text.iloc[i])
    print('---')

Очень большой выбор обуви для всей семьи, по разным ценам)))) Мне магазин очень понравился. Плюс всегда действует акция 2+1.
---
Очень сложно добраться на пляж по сломанной лестнице, рядом порт, с которого постоянной слышен грохот и идет гарь с работающих двигателей кораблей. В воде много стекол и острых предметов. Песок грязный и его практически нет - одни камни. На этот пляж ходить не советую, лучше поехать на Центральный пляж или платный Елисеевский, рядом с аквапарком Лазурный.
---
Вкусное место в центре города.  Разнообразное меню.  Отзывчивый персонал.  Несколько залов разной вместимости дают возможность проводить крупные мероприятия. Вкусная кондитерская на первом этаже.
---
Самый большой плюс это месторасположение, набережная , шикарный вид на море! Красиво, уютно, вот собственно плюсы закончились .. огорчает отношение к посетителям, официанты неприветливые, не здравствуйте вам, не до свидания . Лица недовольные, неприятные, больше не хочется смотреть на такие! Кухня тоже остав

In [ ]:
# Очистка текста
def clear_text(text):
    text = re.sub(r"[Â]+", "'", text)  # Замена Â символов на апостроф.
    text = re.sub(r"#\w+", "", text)  # Удаление хэштегов
    text = re.sub(r"[^\w\s]", "", text)  # Удаление эмодзи и спецсимволов
    text = " ".join(text.split())  # Удаление лишних пробелов
    text = text.lower()  # Приведение к нижнему регистру

    return text


# Очистка
df.loc[:, "text"] = df["text"].apply(clear_text)

In [ ]:
# Вывод строк с индексами от 15 до 23
for i in range(10, 34):
    print(df.text.iloc[i])
    print('---')

очень большой выбор обуви для всей семьи по разным ценам мне магазин очень понравился плюс всегда действует акция 21
---
очень сложно добраться на пляж по сломанной лестнице рядом порт с которого постоянной слышен грохот и идет гарь с работающих двигателей кораблей в воде много стекол и острых предметов песок грязный и его практически нет одни камни на этот пляж ходить не советую лучше поехать на центральный пляж или платный елисеевский рядом с аквапарком лазурный
---
вкусное место в центре города разнообразное меню отзывчивый персонал несколько залов разной вместимости дают возможность проводить крупные мероприятия вкусная кондитерская на первом этаже
---
самый большой плюс это месторасположение набережная шикарный вид на море красиво уютно вот собственно плюсы закончились огорчает отношение к посетителям официанты неприветливые не здравствуйте вам не до свидания лица недовольные неприятные больше не хочется смотреть на такие кухня тоже оставляет желать лучшего в люле кебаб кости попа

In [ ]:
from collections import Counter

def analyze_text(text):
    # Разбиваем текст на слова
    words = text.split()

    # Уникальные слова
    unique_words = set(words)

    # Часто встречающиеся слова
    word_counts = Counter(words)

    return len(unique_words), unique_words, word_counts

# Анализ текста
unique_count, unique_words, word_counts = analyze_text(all_text)

# Вывод результатов
print(f"Количество уникальных слов: {unique_count}")
print(f"Уникальные слова: {unique_words}")